The reception is approaching! With your help, Alfred is now nearly finished with the preparations.

But now there's a problem: the Batmobile has disappeared. Alfred needs to find a replacement, and find it quickly.

Fortunately, a few biopics have been done on Bruce Wayne's life, so maybe Alfred could get a car left behind on one of the movie set, and re-engineer it up to modern standards, which certainly would include a full self-driving option.

But this could be anywhere in the filming locations around the world - which could be numerous.

So Alfred wants your help. Could you build an agent able to solve this task?

> 👉 Find all Batman filming locations in the world, calculate the time to transfer via boat to there, and represent them on a map, with a color varying by boat transfer time. Also represent some supercar factories with the same boat transfer time.

Let's build this!

In [9]:
# !pip install 'smolagents[litellm]' matplotlib geopandas shapely kaleido -q
# !pip install python-dotenv

In [80]:
from huggingface_hub import notebook_login
notebook_login()

In [81]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [82]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple
from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


For the model provider, we use Together AI, one of the new [inference providers on the Hub](https://huggingface.co/blog/inference-providers)!

Regarding the GoogleSearchTool: this requires either having setup env variable `SERPAPI_API_KEY` and passing `provider="serpapi"` or having `SERPER_API_KEY` and passing `provider=serper`.

If you don't have any Serp API provider setup, you can use `DuckDuckGoSearchTool` but beware that it has a rate limit.

In [83]:
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, HfApiModel, VisitWebpageTool

model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

We can start with creating a baseline, simple agent to give us a simple report.

In [84]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [85]:
os.environ["SERPER_API_KEY"] = os.getenv("SERPAPI_API_KEY")

In [86]:
agent = CodeAgent(
    model=model,
    tools=[GoogleSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [88]:
# result = agent.run(task)
task1 = "Find Batman filming locations worldwide aand return them to me as a pandas dataframe"
task2 = "Calculate cargo plane transfer times to Gotham for these locations and return them to me as a pandas dataframe"
task3 = "Find supercar factories with similar transfer times."

response1 = agent.run(task1)
response2 = agent.run(task2)
response3 = agent.run(task3)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find Batman filming locations worldwide aand return them to me as a pandas dataframe                            │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID:
Root=1-67d0677c-079f8d2e15d5cf714003d92a;ed3e60f0-4191-4ec5-b90a-8b0723d9ba4e)

You have exceeded your monthly included credits for Inference Providers.

[Step 1: Duration 0.30 seconds]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 20 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID:
Root=1-67d06783-3ecb880266dc32b44d8b5077;54bc1705-8d50-4559-be7f-b8f92056f0f3)

You have exceeded your monthly included credits for Inference Providers.

[Step 20: Duration 0.14 seconds]

Reached max steps.

[Step 21: Duration 0.24 seconds]

In [46]:
response1

,City,Country,Description
0,![The Batman filming location: St George's Hall,Liverpool],../f/Fantastic-Beasts-St-Georges-ext.jpg
1,The Batman location: The memorial service for ...,St George's Hall,https://commons.wikimedia.org/wiki/File:St._Ge...
2,![The Batman filming location: Lethaby Building,Southampton Row,
3,![The Batman filming location: Bridge of Sighs,Glasgow Necropolis],The-Batman-Glasgow-Necropolis-Bridge-of-Sighs.jpg
4,The Batman location: The Batman and Selina exi...,Glasgow Necropolis | Photograph: [Wikimedia / ...,https://commons.wikimedia.org/wiki/File:Glasgo...
5,![The Batman filming location: Cherry Lane ent...,Anfield Cemetery,The-Batman-Anfield-Cemetery.jpg
6,The Batman location: The Batman and Selina exi...,Anfield Cemetery,https://commons.wikimedia.org/wiki/File:Anfiel...
7,***The Batman* was primarily filmed in the Uni...,although **Chicago was a major influence** on ...,
8,Pattinson is joined by the daughter of pop sta...,**Zoë Kravitz**,https://www.atlasofwonders.com/2018/11/fantast...
9,Welcome to Gotham City and the most complete g...,follow this link to check out all the places [...,https://www.atlasofwonders.com/2024/09/where-w...


In [47]:
response2

,Location,Travel Time (hours)
0,New York City,1.23
1,Los Angeles,6.74
2,Chicago,2.68
3,Houston,4.21


In [48]:
response3

'Lamborghini and Ferrari'

In [50]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task1}
""")

# print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find Batman filming locations worldwide aand return them to me as a pandas dataframe                            │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
```
1. Perform a web search to find websites that list Batman filming locations.
2. Visit the top search results to identify reliable sources of Batman filming locations.
3. For each reliable source, extract the list of Batman movies and their filming locations.
4. For each filming location, visit the source URL to confirm the information.
5. Compile the confirmed filming locations into a structured format, including the movie name, filming location, 
and source URL.
6. Convert the compiled data into a pandas dataframe.
7. Finalize the dataframe to ensure it contains all necessary information and is correctly formatted.
8. Provide the final answer as the pandas dataframe.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  queries = [                                                                                                      
      "Batman filming locations",                                                                                  
      "Batman movie filming locations",                                                                            
      "Where Batman movies were filmed",                                                                           
      "Batman movie locations",                                                                                    
      "Batman filming sites"                                                                                       
  ]                                                                                                                
                                                                                                                   
  for query in queries:                                                                                            
      print(f"Search results for '{query}':")                                                                      
      print(web_search(query=query))                                                                               
      print("\n" + "="*80 + "\n")  # Print separator between queries                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Search results for 'Batman filming locations':
## Search Results
0. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations: Necropolis Cemetery, Glasgow, Scotland, UK (Batman and Selina leaving the cemetery) Helpful•86 1
St. George's Hall, Liverpool, England, UK

1. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow and Chicago.

2. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

Guide to the all the filming locations of The Batman - The 2022 movie with Robert Pattison filmed in Chicago and 
The UK - The City and the Wayne House.

3. [Batman (1989) - Filming & production](https://www.imdb.com/title/tt0096895/locations/)
Source: IMDb

Filming locations ; Knebworth House, Knebworth, Hertfordshire, England, UK. (Wayne Manor; exterior) ; Acton Lane 
Power Station, Acton Lane, Acton, London, England ...

4. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Here are 12 real-world Batman movie locations that you can visit now · St. George's Hall, Liverpool · Two Temple 
Place, London · Wollaton Hall, ...

5. [Dark Knight Rises Tour: See Batman Movie 
Locations](https://www.travelchannel.com/interests/arts-and-culture/photos/see-batman-movie-locations)
Source: Travel Channel

See Batman Movie Locations · Wollaton Hall · Carnegie Mellon University · The Farmiloe Building · Queensboro Bridge
· Wall Street · Mehrangarh Fort · Turda ...

6. [Batman Movies and Filming Locations in Los 
Angeles](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOorAK6Y2wPusH_Q6g4D8gYzNzkrybVqOr_bfQ0m
pxB1wNb8hh_E7)
Source: LA Today

Westwood, Los Angeles, CA. Step into the world of Batman and Robin with a visit to the Kirkeby Center, the location
of a thrilling scene from the 1966 film ...

7. [Batman v Superman: Dawn Of Justice | 2016](https://movie-locations.com/movies/b/Batman-V-Superman.php)
Source: The Worldwide Guide To Movie Locations

Film locations for Batman v Superman: Dawn Of Justice, in Detroit, Chicago, Los Angeles and New Mexico.

8. [The Batman (film)](https://en.wikipedia.org/wiki/The_Batman_(film))
Source: Wikipedia

Filming began in the United Kingdom in January 2020 but was halted by the ... Unlike previous Batman films, The 
Batman focuses on Batman's detective ...

================================================================================

Search results for 'Batman movie filming locations':
## Search Results
0. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations: Necropolis Cemetery, Glasgow, Scotland, UK (Batman and Selina leaving the cemetery) Helpful•86 1
St. George's Hall, Liverpool, England, UK

1. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow and Chicago.

2. [Dark Knight Rises Tour: See Batman Movie 
Locations](https://www.travelchannel.com/interests/arts-and-culture/photos/see-batman-movie-locations)
Source: Travel Channel

See Batman Movie Locations · Wollaton Hall · Carnegie Mellon University · The Farmiloe Building · Queensboro Bridge
· Wall Street · Mehrangarh Fort · Turda Saline.

3. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

The Batman was primarily filmed in the United Kingdom. Most of the recognizable buildings of this new version of 
Gotham City are located i

...

[Step 2: Duration 21.03 seconds| Input tokens: 9,641 | Output tokens: 1,264]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Initialize an empty list to store the data                                                                     
  data = []                                                                                                        
                                                                                                                   
  # Extract data from the visited URLs                                                                             
  # URL 1: https://movie-locations.com/movies/b/The-Batman-2022-2.php                                              
  url1_content = """                                                                                               
  The Batman | Film Locations                                                                                      
                                                                                                                   
  Film locations for The Batman (2022) in Liverpool, London, Glasgow and Chicago.                                  
  - St George's Hall, Liverpool, England, UK                                                                       
  - Royal Liver Building, Liverpool, England, UK                                                                   
  - Bridge of Sighs, Glasgow Necropolis, Glasgow, Scotland, UK                                                     
  - Anfield Cemetery, Liverpool, England, UK                                                                       
  - Michigan Central Station, Detroit, Michigan, USA                                                               
  - Hygrade Deli, Detroit, Michigan, USA                                                                           
  """                                                                                                              
                                                                                                                   
  # URL 2: https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html                                 
  url2_content = """                                                                                               
  Where was The Batman filmed? ALL the Filming Locations in Chicago & The UK                                       
                                                                                                                   
  - Two Temple Place, London, England, UK                                                                          
  - St George's Hall, Liverpool, England, UK                                                                       
  - Wellington's Column, Liverpool, England, UK                                                                    
  - Steble Fountain, Liverpool, England, UK                                                                        
  - Bank of Gotham, St George's Hall, Liverpool, England, UK                                                       
  """                                                                                                              
                                                                                                                   
  # URL 3: https://movie-locations.com/movies/b/Batman-V-Superman.php                                              
  url3_content = """                                                                                               
  Batman v Superman: Dawn Of Justice | 2016                                                                        
                                                         

Execution logs:
                       Movie  \
0          The Batman (2022)   
1          The Batman (2022)   
2          The Batman (2022)   
3          The Batman (2022)   
4          The Batman (2022)   
5          The Batman (2022)   
6          The Batman (2022)   
7          The Batman (2022)   
8          The Batman (2022)   
9          The Batman (2022)   
10         The Batman (2022)   
11  Batman v Superman (2016)   
12  Batman v Superman (2016)   
13  Batman v Superman (2016)   
14  Batman v Superman (2016)   

                                             Location  \
0            St George's Hall, Liverpool, England, UK   
1        Royal Liver Building, Liverpool, England, UK   
2   Bridge of Sighs, Glasgow Necropolis, Glasgow, ...   
3            Anfield Cemetery, Liverpool, England, UK   
4    Michigan Central Station, Detroit, Michigan, USA   
5                Hygrade Deli, Detroit, Michigan, USA   
6               Two Temple Place, London, England, UK   
7            St George's Hall, Liverpool, England, UK   
8         Wellington's Column, Liverpool, England, UK   
9             Steble Fountain, Liverpool, England, UK   
10  Bank of Gotham, St George's Hall, Liverpool, E...   
11   Michigan Central Station, Detroit, Michigan, USA   
12               Hygrade Deli, Detroit, Michigan, USA   
13            Aragon Ballroom, Chicago, Illinois, USA   
14  Weingart Stadium, East Los Angeles College, Lo...   

                                           Source URL  
0   https://movie-locations.com/movies/b/The-Batma...  
1   https://movie-locations.com/movies/b/The-Batma...  
2   https://movie-locations.com/movies/b/The-Batma...  
3   https://movie-locations.com/movies/b/The-Batma...  
4   https://movie-locations.com/movies/b/The-Batma...  
5   https://movie-locations.com/movies/b/The-Batma...  
6   https://www.atlasofwonders.com/2022/04/where-w...  
7   https://www.atlasofwonders.com/2022/04/where-w...  
8   https://www.atlasofwonders.com/2022/04/where-w...  
9   https://www.atlasofwonders.com/2022/04/where-w...  
10  https://www.atlasofwonders.com/2022/04/where-w...  
11  https://movie-locations.com/movies/b/Batman-V-...  
12  https://movie-locations.com/movies/b/Batman-V-...  
13  https://movie-locations.com/movies/b/Batman-V-...  
14  https://movie-locations.com/movies/b/Batman-V-...  

Out - Final answer:                        Movie  \
0          The Batman (2022)   
1          The Batman (2022)   
2          The Batman (2022)   
3          The Batman (2022)   
4          The Batman (2022)   
5          The Batman (2022)   
6          The Batman (2022)   
7          The Batman (2022)   
8          The Batman (2022)   
9          The Batman (2022)   
10         The Batman (2022)   
11  Batman v Superman (2016)   
12  Batman v Superman (2016)   
13  Batman v Superman (2016)   
14  Batman v Superman (2016)   

                                             Location  \
0            St George's Hall, Liverpool, England, UK   
1        Royal Liver Building, Liverpool, England, UK   
2   Bridge of Sighs, Glasgow Necropolis, Glasgow, ...   
3            Anfield Cemetery, Liverpool, England, UK   
4    Michigan Central Station, Detroit, Michigan, USA   
5                Hygrade Deli, Detroit, Michigan, USA   
6               Two Temple Place, London, England, UK   
7            St George's Hall, Liverpool, England, UK   
8         Wellington's Column, Liverpool, England, UK   
9             Steble Fountain, Liverpool, England, UK   
10  Bank of Gotham, St George's Hall, Liverpool, E...   
11   Michigan Central Station, Detroit, Michigan, USA   
12               Hygrade Deli, Detroit, Michigan, USA   
13            Aragon Ballroom, Chicago, Illinois, USA   
14  Weingart Stadium, East Los Angeles College, Lo...   

                                           Source URL  
0   https://movie-locations.com/movies/b/The-Batma...  
1   https://movie-locations.com/movies/b/The-Batma...  
2   https://movie-locations.com/movies/b/The-Ba

[Step 3: Duration 23.97 seconds| Input tokens: 38,116 | Output tokens: 2,882]

In [51]:
detailed_report

,Movie,Location,Source URL
0,The Batman (2022),"St George's Hall, Liverpool, England, UK",https://movie-locations.com/movies/b/The-Batma...
1,The Batman (2022),"Royal Liver Building, Liverpool, England, UK",https://movie-locations.com/movies/b/The-Batma...
2,The Batman (2022),"Bridge of Sighs, Glasgow Necropolis, Glasgow, ...",https://movie-locations.com/movies/b/The-Batma...
3,The Batman (2022),"Anfield Cemetery, Liverpool, England, UK",https://movie-locations.com/movies/b/The-Batma...
4,The Batman (2022),"Michigan Central Station, Detroit, Michigan, USA",https://movie-locations.com/movies/b/The-Batma...
5,The Batman (2022),"Hygrade Deli, Detroit, Michigan, USA",https://movie-locations.com/movies/b/The-Batma...
6,The Batman (2022),"Two Temple Place, London, England, UK",https://www.atlasofwonders.com/2022/04/where-w...
7,The Batman (2022),"St George's Hall, Liverpool, England, UK",https://www.atlasofwonders.com/2022/04/where-w...
8,The Batman (2022),"Wellington's Column, Liverpool, England, UK",https://www.atlasofwonders.com/2022/04/where-w...
9,The Batman (2022),"Steble Fountain, Liverpool, England, UK",https://www.atlasofwonders.com/2022/04/where-w...


In [57]:
model = HfApiModel(
    "Qwen/Qwen2.5-Coder-32B-Instruct", provider="together", max_tokens=8096
)

web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(provider="serper"),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

The manager agent will need to do some mental heavy lifting.

So we give it the stronger model DeepSeek-R1, and add a planning_interval to the mix

In [58]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    final_answer
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=HfApiModel("deepseek-ai/DeepSeek-R1", provider="together", max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

In [59]:
manager_agent.visualize()

CodeAgent | deepseek-ai/DeepSeek-R1
├── ✅ Authorized imports: ['geopandas', 'plotly', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | Qwen/Qwen2.5-Coder-32B-Instruct
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a google web search for  │ query (`string`): The search      │
            │                             │ your query then returns a string  │ query to perform.                 │
            │                             │ of the top search results.        │ filter_year (`integer`):          │
            │                             │                                   │ Optionally restrict results to a  │
            │                             │                                   │ certain year                      │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                             │ distance.     

In [69]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!
Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W).                                                                             │
│ Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in     │
│ total.                                                                                                          │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ HfApiModel - deepseek-ai/DeepSeek-R1 ──────────────────────────────────────────────────────────────────────────╯

[Step 1: Duration 0.11 seconds| Input tokens: 1,382 | Output tokens: 1,142]

HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID: Root=1-67cfb084-2aa8f7fe5989232d408893b5;fcde101d-6090-4a14-975c-39f4fcbe2d25)

You have exceeded your monthly included credits for Inference Providers. Pay-as-you-go above your included PRO quota will be available soon.

I don’t know how that went in your run, but in mine, the manager agent skilfully divided tasks given to the web agent in 1. Search for Batman filming locations, then 2. Find supercar factories, before aggregating the lists and plotting the map.

Let’s see what the map looks like by inspecting it directly from the agent state:

In [64]:
manager_agent.python_executor.state["fig"]

KeyError: 'fig'